<a href="https://colab.research.google.com/github/Qureshihasaan/Summarization_and_Database_Chatbot/blob/main/summarization_and_database_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%pip install -q -U psycopg psycopg-pool langgraph langgraph-checkpoint-postgres langchain_google_genai langsmith

In [16]:
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")

In [17]:
import os

os.environ["LANGCHAIN_API_KEY"] = userdata.get("langsmith_api_key")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_PROJECT'] = "Postgress_db_chatbot"

In [ ]:
print(gemini_api_key)


In [19]:
DB_URL = userdata.get("DATABASE_URL")

In [ ]:
print(DB_URL)

In [21]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

connection_kwargs = {"autocommit": True, "prepare_threshold":0}

pool = ConnectionPool(conninfo=DB_URL, max_size=20 , kwargs=connection_kwargs )

checkpointer = PostgresSaver(pool)
checkpointer.setup()

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key =  gemini_api_key)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State) -> State:

    # Get summary if it exists
    summary = state.get("summary", "")
    print(f"Using summary: {summary}")

    # If there is summary, then we add it
    if summary:

        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]

    else:
        messages = state["messages"]

    response = llm.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State) -> State:
    print(f"Messages before summarizing: {len(state['messages'])}")
    # First, we get any existing summary
    summary = state.get("summary", "")
    print(f"Existing summary: {summary}")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"


    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    # Summarization logic
    print(f"New summary: {response.content}")

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    print(f"Messages after truncation: {len(delete_messages)}")
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State) -> State:

    """Return the next node to execute."""

    messages = state["messages"]
    print(f"Message count: {len(messages)}")
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"

    # Otherwise we can just end
    return END


In [23]:
from langgraph.graph import StateGraph, START , END

builder = StateGraph(State)

builder.add_node("conversation", call_model)
builder.add_node(summarize_conversation)


builder.add_edge(START , "conversation")
builder.add_conditional_edges("conversation", should_continue)
builder.add_edge("summarize_conversation", END)


graph =  builder.compile(checkpointer=checkpointer)

In [24]:
config = {"configurable" : {"thread_id" : "1"}}


input_message = HumanMessage(content="what's i like playing")
output = graph.invoke({"messages":input_message}, config)
for m in output["messages"][-1:]:
  m.pretty_print()


Using summary: The conversation began with the user asking for their name, to which I responded that I don't have access to personal information.  The user then stated they enjoy playing cricket.  Finally, the user asked for a summary of our conversation, which is this summary.
Message count: 4
================================== Ai Message ==================================

Based on our previous conversation, you mentioned that you like playing cricket.


In [25]:
graph_state = graph.get_state(config)

graph_state

StateSnapshot(values={'messages': [HumanMessage(content="what's my last conversation", additional_kwargs={}, response_metadata={}, id='608402df-0f66-448e-a634-dcffba46c18f'), AIMessage(content="Our last conversation was about your enjoyment of playing cricket.  Before that, you asked me what your name was, and I explained that I don't have access to that information.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-34c4dbfc-d785-4dee-86a4-baf5f79ab9b4-0', usage_metadata={'input_tokens': 88, 'output_tokens': 38, 'total_tokens': 126, 'input_token_details': {'cache_read': 0}}), HumanMessage(content="what's i like playing", additional_kwargs={}, response_metadata={}, id='45859edb-a832-422b-b03d-10618ecd3997'), AIMessage(content='Based on our previous conversation, you mentioned that you like playing cricket.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_re

In [26]:
config = {"configurable" : {"thread_id" : "1"}}


input_message = HumanMessage(content="what's my last conversation")
output = graph.invoke({"messages":input_message}, config)
for m in output["messages"][-1:]:
  m.pretty_print()

Using summary: The conversation began with the user asking for their name, to which I responded that I don't have access to personal information.  The user then stated they enjoy playing cricket.  Finally, the user asked for a summary of our conversation, which is this summary.
Message count: 6
================================== Ai Message ==================================

Our last conversation was short. You asked me what you like playing, and I responded that you mentioned liking cricket in our previous conversation.


In [28]:
graph_state= graph.get_state(config)

In [29]:
graph_state

StateSnapshot(values={'messages': [HumanMessage(content="what's my last conversation", additional_kwargs={}, response_metadata={}, id='608402df-0f66-448e-a634-dcffba46c18f'), AIMessage(content="Our last conversation was about your enjoyment of playing cricket.  Before that, you asked me what your name was, and I explained that I don't have access to that information.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-34c4dbfc-d785-4dee-86a4-baf5f79ab9b4-0', usage_metadata={'input_tokens': 88, 'output_tokens': 38, 'total_tokens': 126, 'input_token_details': {'cache_read': 0}}), HumanMessage(content="what's i like playing", additional_kwargs={}, response_metadata={}, id='45859edb-a832-422b-b03d-10618ecd3997'), AIMessage(content='Based on our previous conversation, you mentioned that you like playing cricket.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_re

In [30]:
config = {"configurable" : {"thread_id" : "1"}}


input_message = HumanMessage(content="can you describe about vector ")
output = graph.invoke({"messages":input_message}, config)
for m in output["messages"][-1:]:
  m.pretty_print()

Using summary: The conversation began with the user asking for their name, to which I responded that I don't have access to personal information.  The user then stated they enjoy playing cricket.  Finally, the user asked for a summary of our conversation, which is this summary.
Message count: 8
Messages before summarizing: 8
Existing summary: The conversation began with the user asking for their name, to which I responded that I don't have access to personal information.  The user then stated they enjoy playing cricket.  Finally, the user asked for a summary of our conversation, which is this summary.
New summary: The conversation began with the user asking for their name, to which I responded that I don't have access to personal information.  The user then stated they enjoy playing cricket.  The user then again requested a summary of the conversation.  Following this, the user asked for a description of vectors. I provided a detailed explanation of vectors, including their representat

In [31]:
graph_state = graph.get_state(config)
graph_state

StateSnapshot(values={'messages': [HumanMessage(content='can you describe about vector ', additional_kwargs={}, response_metadata={}, id='a4c4a113-2965-45e2-b0d0-b0c86f627e7e'), AIMessage(content='A vector is a quantity that has both magnitude (size) and direction.  This is in contrast to a scalar, which only has magnitude.\n\nHere\'s a breakdown of key aspects of vectors:\n\n* **Representation:** Vectors are often represented visually as arrows. The length of the arrow represents the magnitude, and the direction of the arrow represents the direction of the vector.  They can also be represented mathematically using coordinates (e.g., in two dimensions as (x, y) or in three dimensions as (x, y, z)).\n\n* **Magnitude:**  This is the "size" or "length" of the vector.  For example, if a vector represents displacement, the magnitude would be the distance.  Mathematically, the magnitude is calculated using the Pythagorean theorem (or its higher-dimensional equivalent).\n\n* **Direction:** Th